In [137]:
import pandas as pd
import json
import nltk
from nltk.tokenize import PunktSentenceTokenizer
from nltk.tokenize import regexp_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
# LDA
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

spanish_sw = set(stopwords.words("spanish"))

- **Ejercicio 1:** Dado un fichero en formato JSON con noticias se quiere obtener los diferentes topics utilizando LDA. En este primer ejercicio se proporciona el notebook completo hacerlo, los pasos son los siguientes:
    - Lectura del fichero y carga de datos.
    - Preprocesamiento básico de los textos.
    - Entrenamiento del algoritmo Latent Dirichlet Allocation de sklearn. Se recomienda ver la documentación de la librería para ver los diferentes parámetros del algoritmo.
    - Visualización de resultados.
    - Evaluación de los resultados.  

    Se pide ejecutar el código proporcionado y realizar ajustes sobre él para ver cómo
    cambian los resultados:
    - Cambiar el número de topics, ahora está a 2, pero revisando el fichero con las
    noticias, en realidad serían más.
    - Cambiar el preprocesamiento de los textos, eliminando stopwords al menos y lo
    que se considere para ver si mejoran los resultados.  
    
    ¿Cómo impacta en los resultados los diferentes cambios de los apartados anteriores?

In [138]:
# Carga del fichero "Noticias.json"
with open("data/Noticias.json", "r") as f:
    noticias = json.load(f)

df_noticias = pd.DataFrame(noticias)
df_noticias.head()

,Title,TextContent
0,Suspendido el partido Villarreal-Espanyol por ...,El temporal de lluvia y nieve afecta a áreas d...
1,Reino Unido y otros países aliados de Ucrania ...,Los países europeos de la OTAN y Canadá han de...
2,Los premios Oscar dan la gloria al cine indie ...,¿Qué premia exactamente Hollywood y su industr...
3,"Emilia Pérez, Karla Sofía Gascón, Demi Moore y...","Fue Beckett el que, en un arrebato no precisam..."
4,La Aemet retira también el aviso rojo por fuer...,La Agencia Estatal de Meteorología (Aemet) ha ...


In [139]:
# Preprocesamiento de textos
def text_preprocessing(text):
    sent_tokenized = PunktSentenceTokenizer().tokenize(text)
    word_tokenized = [regexp_tokenize(sent, r"[a-zA-Z0-9áéíóúÁÉÍÓÚ]{3,}") for sent in sent_tokenized]
    word_tokenized = [[w for w in sent if w.lower() not in spanish_sw] for sent in word_tokenized]
    lemmatized = [" ".join([WordNetLemmatizer().lemmatize(w) for w in sent]) for sent in word_tokenized][0]
    return lemmatized

preprocessed_titles = df_noticias["Title"].apply(text_preprocessing)
preprocessed_texts = df_noticias["TextContent"].apply(text_preprocessing)
df_noticias_preprocessed = pd.concat((preprocessed_titles, preprocessed_texts), axis=1)
print("DataFrame de noticias aplicando eliminación de puntuación y stopwords y aplicando lematización:")
print(f"Tamaño del DF: {df_noticias_preprocessed.shape}")
df_noticias_preprocessed.head(5)

DataFrame de noticias aplicando eliminación de puntuación y stopwords y aplicando lematización:
Tamaño del DF: (12, 2)


,Title,TextContent
0,Suspendido partido Villarreal Espanyol emergen...,temporal lluvia nieve afecta áreas diez provin...
1,Reino Unido países aliados Ucrania comprometen...,países europeos OTAN Canadá decidido aumentar ...
2,premios Oscar dan gloria cine indie castigan v...,premia exactamente Hollywood industria disting...
3,Emilia Pérez Karla Sofía Gascón Demi Moore dec...,Beckett arrebato precisamente entusiasmo dijo ...
4,Aemet retira aviso rojo fuertes lluvias Castel...,Agencia Estatal Meteorología Aemet levantado n...


In [140]:
# Generación de la BoW
vectorizer = CountVectorizer()
bow_encoded = vectorizer.fit_transform(df_noticias_preprocessed["TextContent"].tolist())
vocabulary = vectorizer.vocabulary_
dictionary = vectorizer.get_feature_names_out()
print("Vocabulario:")
print(vocabulary)
print("Matriz rasgos-documentos:")
print(bow_encoded.toarray())

Vocabulario:
{'temporal': 195, 'lluvia': 107, 'nieve': 132, 'afecta': 9, 'áreas': 210, 'diez': 63, 'provincias': 167, 'repartidas': 176, 'fachada': 88, 'mediterránea': 116, 'centro': 41, 'peninsular': 149, 'especialmente': 78, 'ronda': 182, 'málaga': 126, 'interior': 98, 'sur': 192, 'castellón': 38, 'residentes': 177, 'recibido': 173, 'móviles': 128, 'alertas': 11, 'agencia': 10, 'estatal': 82, 'meteorología': 119, 'aemet': 8, 'activado': 5, 'aviso': 23, 'rojo': 181, 'riesgo': 180, 'extremo': 87, 'lluvias': 108, 'persistentes': 151, 'largo': 103, 'tarde': 194, 'países': 147, 'europeos': 85, 'otan': 141, 'canadá': 35, 'decidido': 55, 'aumentar': 21, 'ayuda': 24, 'militar': 121, 'ucrania': 200, 'sanciones': 184, 'económicas': 74, 'rusia': 183, 'gobierno': 91, 'kiev': 101, 'pueda': 169, 'negociar': 130, 'moscú': 124, 'posición': 157, 'fortaleza': 90, 'tal': 193, 'declarado': 56, 'domingo': 68, 'primer': 162, 'ministro': 123, 'británico': 34, 'keir': 100, 'starmer': 191, 'después': 61, 'cu

In [141]:
# Entrenamiento del algoritmo Latent Dirichlet Allocation
lda = LatentDirichletAllocation(
    n_components=4,
    doc_topic_prior=0.6,
    topic_word_prior=0.9,
    max_iter=25,
    learning_method="online",
    evaluate_every=1,
    n_jobs=-1,
    verbose=True
    )
lda.fit(bow_encoded)

iteration: 1 of max_iter: 25, perplexity: 430.9137
iteration: 2 of max_iter: 25, perplexity: 394.9068
iteration: 3 of max_iter: 25, perplexity: 369.9914
iteration: 4 of max_iter: 25, perplexity: 353.9579
iteration: 5 of max_iter: 25, perplexity: 343.4075
iteration: 6 of max_iter: 25, perplexity: 336.3051
iteration: 7 of max_iter: 25, perplexity: 331.4102
iteration: 8 of max_iter: 25, perplexity: 327.9622
iteration: 9 of max_iter: 25, perplexity: 325.4855
iteration: 10 of max_iter: 25, perplexity: 323.6760
iteration: 11 of max_iter: 25, perplexity: 322.3341
iteration: 12 of max_iter: 25, perplexity: 321.3256
iteration: 13 of max_iter: 25, perplexity: 320.5588
iteration: 14 of max_iter: 25, perplexity: 319.9695
iteration: 15 of max_iter: 25, perplexity: 319.5122
iteration: 16 of max_iter: 25, perplexity: 319.1544
iteration: 17 of max_iter: 25, perplexity: 318.8721
iteration: 18 of max_iter: 25, perplexity: 318.6477
iteration: 19 of max_iter: 25, perplexity: 318.4681
iteration: 20 of max_

LatentDirichletAllocation(doc_topic_prior=0.6, evaluate_every=1,
                          learning_method='online', max_iter=25, n_components=4,
                          n_jobs=-1, topic_word_prior=0.9, verbose=True)

In [ ]:
# Distribución de documentos por tópico
doc_topics = lda.transform(bow_encoded)
# Distribución de palabras por tópico
topics = lda.components_

# Número de palabras top por tópico
no_top_words = 10
# Número de documentos top por tópico
no_top_documents = 2

for topic_idx, topic in enumerate(topics):
    print(f"Tópico {topic_idx+1}:")
    top_words_idx = topics.argsort()[:-no_top_words-1:-1]
    top_words = [dictionary[i] for i in top_words_idx]
    top_probs = [topic[i] for i in top_words_idx]

    for word, prob in zip(top_words, top_probs):
        print(f"    Palabra: {word}; Probabilidad: {prob}")
    print()
    

Tópico 1:
    Palabra: ['diez' 'máximo' 'durar' 'tarde' 'peninsular' 'nevará' 'seis' 'recibido'
 'laboratorios' 'presión' 'martes' 'espejo' 'bacteria' 'cayeran' 'aunque'
 'provincias' 'afecta' 'castellón' 'hollywood' 'áreas' 'científicos'
 'ordenado' 'península' 'metro' 'posibles' 'extremo' 'centro' 'lluvias'
 'pausa' 'cerrar' 'acuerdo' 'residentes' 'lluvia' 'especialmente' 'largo'
 'nunca' 'casa' 'persistentes' 'tenerife' 'acariciado' 'entrega'
 'presidente' 'voluntad' 'torno' 'móviles' 'arreciando' 'nieve' 'además'
 'demuestre' 'paz' '180' 'temporal' 'deshacerse' 'construir' '2024' 'idea'
 'catastróficas' 'aviso' 'agencia' 'dólares' 'noche' 'ronda'
 'estadounidense' 'estrecho' 'interior' 'palma' 'principio' 'fachada'
 'activado' 'iba' 'alertas' 'academia' 'aparejos' 'según' 'proyecto'
 'horas' 'día' 'merecido' 'destacada' 'posibilidad' 'optado' 'málaga'
 'val' 'universitat' 'oasis' 'querer' 'biólogos' 'animales' 'levantado'
 'seguirá' 'volodímir' 'nueve' 'zelenski' 'cineasta' 'sintét

In [143]:
vectorizer.get_feature_names_out()

array(['180', '2024', 'abundancia', 'academia', 'acariciado', 'activado',
       'acuerdo', 'además', 'aemet', 'afecta', 'agencia', 'alertas',
       'alguna', 'aliados', 'animales', 'antigua', 'aparejos', 'aquello',
       'arrebato', 'arreciando', 'atención', 'aumentar', 'aunque',
       'aviso', 'ayuda', 'ayuntamiento', 'bacteria', 'bares', 'bebida',
       'beckett', 'bien', 'biología', 'biólogos', 'blanca', 'británico',
       'canadá', 'canarias', 'casa', 'castellón', 'catastróficas',
       'cayeran', 'centro', 'cerrar', 'científicos', 'cierren', 'cine',
       'cineasta', 'confirmado', 'consecuencias', 'considera',
       'construir', 'cuadrado', 'cuatro', 'cumbre', 'debido', 'decidido',
       'declarado', 'demuestre', 'desechados', 'deshacerse', 'despejar',
       'después', 'destacada', 'diez', 'dijo', 'dirigida', 'distingue',
       'dolby', 'domingo', 'donald', 'dos', 'durar', 'día', 'dólares',
       'económicas', 'entrega', 'entusiasmo', 'espa', 'especialmente',
       '

In [144]:
vectorizer.vocabulary_

{'temporal': 195,
 'lluvia': 107,
 'nieve': 132,
 'afecta': 9,
 'áreas': 210,
 'diez': 63,
 'provincias': 167,
 'repartidas': 176,
 'fachada': 88,
 'mediterránea': 116,
 'centro': 41,
 'peninsular': 149,
 'especialmente': 78,
 'ronda': 182,
 'málaga': 126,
 'interior': 98,
 'sur': 192,
 'castellón': 38,
 'residentes': 177,
 'recibido': 173,
 'móviles': 128,
 'alertas': 11,
 'agencia': 10,
 'estatal': 82,
 'meteorología': 119,
 'aemet': 8,
 'activado': 5,
 'aviso': 23,
 'rojo': 181,
 'riesgo': 180,
 'extremo': 87,
 'lluvias': 108,
 'persistentes': 151,
 'largo': 103,
 'tarde': 194,
 'países': 147,
 'europeos': 85,
 'otan': 141,
 'canadá': 35,
 'decidido': 55,
 'aumentar': 21,
 'ayuda': 24,
 'militar': 121,
 'ucrania': 200,
 'sanciones': 184,
 'económicas': 74,
 'rusia': 183,
 'gobierno': 91,
 'kiev': 101,
 'pueda': 169,
 'negociar': 130,
 'moscú': 124,
 'posición': 157,
 'fortaleza': 90,
 'tal': 193,
 'declarado': 56,
 'domingo': 68,
 'primer': 162,
 'ministro': 123,
 'británico': 34,
 